In [51]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [6]:
import modules.experiment.events as expevents
import modules.experiment.fov as expfov
import modules.utils as utils
import modules.plots.experimental_data as expplots
import modules.eas_reconstruction as eas

from modules_reloader import reloader

relmod = reloader(expevents, expfov, utils, expplots, eas)

In [9]:
relmod()

In [37]:
processor = expevents.EventProcessor(N=45, verbosity=1, load_rir=False, min_signal_significance=4)
L = 10
N = processor.N

In [38]:
event_id = 10675
event = expevents.Event(event_id)

In [39]:
fov = expfov.PmtFov.for_event(event_id)

In [43]:
n_samples, has_signal = processor.read_reconstruction_marginal_sample_per_channel(event.id_, parameter='n')
significant = processor.read_signal_significances(event.id_)[has_signal] > processor.min_signal_significance
axis_theta, axis_phi, inplane = processor.reconstruct_eas_angle(event)

channels = np.arange(109)

channels = channels[has_signal][significant][inplane]
n_samples = n_samples[:, significant][:, inplane]

In [104]:
n_samples = np.round(n_samples).astype(int)

In [129]:
n_means = np.zeros((n_samples.shape[0]))
n_stds = np.zeros((n_samples.shape[0]))

for i, n_i_sample in enumerate(n_samples.T):
    n_i_sample_round = np.round(n_i_sample).astype(int)
    p_obs = np.bincount(n_i_sample_round[n_i_sample_round > 0])
    n_obs = np.arange(p_obs.size)
    n_obs = n_obs[p_obs > 0]
    p_obs = p_obs[p_obs > 0]

    # lmb_min = max(n_obs.min() / 2, 0)
    lmb_query = np.linspace(0.0, 2 * n_obs.max(), 1000, dtype=float)
    p_obs = p_obs / np.sum(p_obs)

    lmb_pdf = np.zeros_like(lmb_query)
    for n, p in zip(n_obs, p_obs):
        lmb_pdf += p*scipy.stats.poisson.pmf(n, lmb_query)

    lmb_step = lmb_query[1] - lmb_query[0]
    n_final_mean = np.sum(lmb_query * lmb_pdf * lmb_step)
    n_sq_final_mean = np.sum(lmb_query**2 * lmb_pdf * lmb_step)
    n_final_std = np.sqrt(n_sq_final_mean - n_final_mean ** 2)

    n_means[i] = n_final_mean
    n_stds[i] = n_final_std

    # plt.hist(n_i_sample, density=True)
    # plt.plot(lmb_query, lmb_pdf)
    # break

    # print(f'before poisson: {n_i_sample.mean()} +/- {n_i_sample.std()}')
    # print(f'after poisson: {n_final_mean} +/- {n_final_std}')


In [13]:
axis_x, axis_y, _, _ = processor.read_eas_geometry(event.id_)

In [14]:
axis_x_proj, axis_y_proj = eas.project_on_shower_plane(np.array([axis_x]), np.array([axis_y]), axis_theta, axis_phi)

In [16]:
fov_x_proj, fov_y_proj = eas.project_on_shower_plane(fov.x, fov.y, axis_theta, axis_phi)

fov_x_proj -= axis_x_proj
fov_y_proj -= axis_y_proj

In [24]:
from scipy.interpolate import griddata
from tqdm import trange

FOV_proj = np.zeros((109, fov.side, fov.side))

for i_ch in trange(109):
    fov_grid = fov.grid()
    x_ch, y_ch = np.meshgrid(fov_grid, fov_grid)
    x_ch = x_ch.flatten()
    y_ch = y_ch.flatten()
    x_ch_proj, y_ch_proj = eas.project_on_shower_plane(x_ch, y_ch, axis_theta, axis_phi)
    FOV_ch = np.squeeze(fov.FOV[i_ch, :, :]).flatten()
    FOV_proj[i_ch, :, :] = griddata(
        utils.concat_vectors_as_cols(x_ch_proj, y_ch_proj),
        FOV_ch,
        utils.concat_vectors_as_cols(x_ch, y_ch),
        method='cubic',
        fill_value=0.0,
    ).reshape(fov.side, fov.side)


fov_proj = expfov.PmtFov(
    FOVc=np.concatenate((np.expand_dims(fov_x_proj, 1), np.expand_dims(fov_y_proj, 1)), axis=1),
    FOV=FOV_proj
)

100%|██████████| 109/109 [00:06<00:00, 16.30it/s]


In [31]:
# see https://arxiv.org/pdf/astro-ph/0511215.pdf

def Q(x, y, P, Qkn):
    R = np.sqrt(x ** 2 + y ** 2)
    R0 = 10 ** (2.95 - 0.245 * P)
    Rkn = 155 - 13 * P
    b = 1.19 + 0.23 * P
    Q_ = np.zeros_like(R)
    Q_[R < Rkn] = Qkn * np.exp((Rkn - R) * (1 + 3 / (R + 3)) / R0)
    Q_[R >= Rkn] = Qkn * (Rkn / R) ** b
    return Q_

def theory_n_phels_in_ch(i_ch, P, Qkn):
    # see 13_pmt_quantum_efficiencies.ipynb for magic numbers
    #           Hamamatsu                                ФЭУ84-3
    coeff = 1 / 2.943031538444593 if i_ch == 0 else 1 / 2.5745613410569366
    fov_grid = fov_proj.grid()
    x, y = np.meshgrid(fov_grid, fov_grid)
    x = x.flatten() + fov_proj.FOVc[i_ch, 0]
    y = y.flatten() + fov_proj.FOVc[i_ch, 1]
    f = np.squeeze(fov_proj.FOV[i_ch, :, :]).flatten()
    x = x[f > 1e-15]
    y = y[f > 1e-15]
    f = f[f > 1e-15]
    Q_vals = Q(x, y, P, Qkn)
    return coeff * np.sum(Q_vals * f) * fov_proj.step ** 2


In [32]:
from scipy.optimize import curve_fit

curve_fit(
    theory_n_phels_in_ch,
    
)